## Convert behavior data

In [9]:
from numpy import concatenate, cumsum, convolve, ones
from pandas import read_csv, concat
from glob import glob
from os.path import exists

###Set directory and session information

In [10]:
directory = '/tier2/freeman/Nick/mVR/sessions'

Check if behavior data exists

In [11]:
key = '000133'
path = directory + '/' + key
print path
print exists(path)

/tier2/freeman/Nick/mVR/sessions/000133
True


###Load trial data

In [12]:
files = glob(path + '/behavior/trial-*.csv')
files.sort()
trials = [read_csv(x) for x in files]

Create session table

In [13]:
session = concat(trials)
session.reset_index(inplace=True)
session.rename(columns={'index': 'tick'}, inplace=True)

Add global time

In [14]:
trial = session.groupby(['number']).last()
offsets = concatenate(([0],trial.time.values))
offsets = cumsum(offsets)
session['allTime'] = session.time + offsets[session.number]

Add filtered speed

In [15]:
speed = (session.xSpeed.pow(2)+session.ySpeed.pow(2)).pow(.5)
session['speed'] = convolve(speed,ones(50)/50, mode='same')    

### Save full-length session

In [16]:
session.to_csv(path + '/behavior/session.csv')